In [1]:
# Import dependencies
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

## Database Setup

In [2]:
# Database Setup
# Create engine to hawaii.sqlite
engine = create_engine("sqlite:///..\Resources\hawaii.sqlite")

In [3]:
# Reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

C:\Users\User1\AppData\Local\Temp\ipykernel_11608\1603037778.py:3: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [4]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [5]:
## Create a session
session = Session(engine)

## Flask setup

In [6]:
app = Flask(__name__)

## Flask Routes

In [7]:
# Homepage with available api routes
def home():
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/temp/start<start><br/>"
        f"/api/v1.0/temp/start/end<br/>"
        f"/api/v1.0/2016-08-23/2017-08-23"
        f"<p>'start' and 'end' date should be in the format MMDDYYYY.</p>"
    )

In [8]:
@app.route("/api/v1.0/precipitation")

# Create the precipitation() function
def precipitation():
	# Calculate the date one year ago from the most recent date
	prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

	# Query: get date and precipitation for prev_year
	precipitation = session.query(Measurement.date,Measurement.prcp) .\
		filter(Measurement.date >= prev_year).all()
		
	# Create dictionary w/ jsonify()--format results into .JSON
	precip = {date: prcp for date, prcp in precipitation}
	return jsonify(precip)

In [9]:
# Stations API route
@app.route("/api/v1.0/stations")
def stations():
    """Return a JSON list of stations from the dataset."""
    station_results = session.query(Station.station).all()

    session.close()
    
    station_list = [station[0] for station in station_results]
    return jsonify(station_list)

In [10]:
# Tobs API route
@app.route("/api/v1.0/tobs")
def tobs():
    """Query the dates and temperature observations of the most-active station for the previous year of data."""
    most_active_station = "USC00519281"
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    tobs_results = session.query(Measurement.date, Measurement.tobs)\
        .filter(Measurement.station == most_active_station)\
        .filter(Measurement.date >= prev_year).all()

    session.close()
    
    temperature_data = [{'date': date, 'temperature': tobs} for date, tobs in tobs_results]
    return jsonify(temperature_data)

In [11]:
# Specified start API route
@app.route("/api/v1.0/<start>")
def temperature_start(start):
    """Return a JSON list of the minimum, average, and maximum temperature for a specified start date."""
    start_results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
        .filter(Measurement.date >= start).all()
    temperature_data = {
        'TMIN': start_results[0][0],
        'TAVG': start_results[0][1],
        'TMAX': start_results[0][2]
    }
    return jsonify(temperature_data)


In [12]:
# Specified start/end API route
@app.route("/api/v1.0/<start>/<end>")
def temperature_start_end(start, end):
    """Return a JSON list of the minimum, average, and maximum temperature for a specified start-end date range."""
    start_end_results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
        .filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    temperature_data = {
        'TMIN': start_end_results[0][0],
        'TAVG': start_end_results[0][1],
        'TMAX': start_end_results[0][2]
    }
    return jsonify(temperature_data)
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\User1\anaconda3\envs\dev\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
